Import libraries

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_digits
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler

**Upload Data**

We upload the data in a pandas dataframe.

In [ ]:
# Path to dataset with dropped NaN values
local_dropped_path = r'../../data/2015_cleaned_droppedNaN.csv'
# Path to dataset with imputed NaN values
local_imputed_path = '../../data/2015_cleaned_imputedNaN.csv'
#df = pd.read_csv(local_imputed_path)
df = pd.read_csv(local_dropped_path)

# Pre-Processing Data

In [ ]:
df.head()

In [ ]:
df.shape

**Check empty values**

In [ ]:
print(round((((df.isnull().sum()).sum() / np.product(df.shape)) * 100), 2))

We recommend to reduce the database for testing purposes. Please uncomment the code below to reduce the dataset,
otherwise the time required to run the code increases considerably or the computer might crushed. We recommend to run
the complete dataset in google-CoLab since it provides more computer power.

In [ ]:
df = df.loc[0:2500]
df.shape

**Split Data**

In [ ]:
X = df.drop(columns=['HeartDiseaseorAttack'])
y = np.array(df['HeartDiseaseorAttack'])

**Balance Data**

We performed undersampling because our target is not balanced.

In [ ]:
rus = RandomUnderSampler()
X_resampled, y_resampled = rus.fit_resample(X, y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size = 0.2)

In [ ]:
y_train.shape

In [ ]:
X_train.shape

# KNN

We have previously tested the performance with StandardScaler() and MinMaxScaler() and we have obtained, that the performance is very similar or identical. Since the data has been already scaled with MinMaxScaler() for the KNN-Imputer, we will not scaled the data again here.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

Note: We have chosen a range of odd numbers for the k, to try to avoid "ties" as much as possible.

### optimize KNN with RandomizedSearchCV

We tried to optimized the KNN using RandomizedSearchCV. We will try to get the best number of neighbors.


In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
knn = KNeighborsClassifier()

We generate a list of possible values for k, which are all odd to avoid possible "ties".

In [ ]:
k = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]

In [ ]:
from sklearn.metrics import DistanceMetric
params = {'n_neighbors' : k, 'metric': ['manhattan','euclidean', 'minkowski']}

cv=5 means cross validation with 5 folds

In [ ]:
random_search = GridSearchCV(knn, params, cv=5)
random_search.fit(X_train, y_train)

Call score_samples on the estimator with the best found parameters.

In [ ]:
random_search.best_params_

In [ ]:
best_k = random_search.best_params_.get('n_neighbors')
print(best_k)

In [ ]:
metric =random_search.best_params_.get('metric')

Check if we get the same or very similar result:

In [ ]:
accuracy_list =[]
for i in range(100):

  knn = KNeighborsClassifier(n_neighbors=best_k, metric = metric)
  knn.fit(X_train, y_train)
  prediction = knn.predict(X_test)
  accuracy = accuracy_score(y_test, prediction)
  accuracy_list.append(accuracy)

print("Average Accuracy Score is: ", np.mean(accuracy_list))

# Other Evaluations

## Confusion Matrix

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(knn, X_test, y_test)

We have gotten a relatively high number of false negatives.

In [ ]:
collector = []
for k in range(1, 20, 2):
  accuracy_list = []
  for j in range(1, 10): # I run 10 times and take the mean accuracy
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    prediction = knn.predict(X_test)
    accuracy = accuracy_score(y_test, prediction)
    accuracy_list.append(accuracy)
  collector.append({"k" : k,
                    "accuracy": round(np.mean(accuracy_list), 3).astype('float64')})

accuracy_scores_minMax_df = pd.DataFrame(collector)

plt.plot(accuracy_scores_minMax_df['k'], accuracy_scores_minMax_df['accuracy'])
plt.grid(True)
plt.show()